In [1]:
from urllib.parse import urlparse, urljoin, ParseResult
from crawler import PageFetcher, Scheduler
from collections import OrderedDict
from crawler.domain import Domain
import seaborn as sns
from matplotlib import pyplot as plt
import time
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
seeds = ["http://www.terra.com.br/", "https://www.uol.com.br/", "https://www.bbc.com/"]

In [3]:
studies = list()
results = list()

In [4]:
n_threads = list(np.arange(1, 20, step=5)) + list(np.arange(30, 100, step=20))
for n in n_threads:
    studies.append({"n_threads": n})

In [ ]:
for study in tqdm(studies):
    t = time.time()
    scheduler = Scheduler("batata-bot", page_limit=100, depth_limit=6, arr_urls_seeds=[urlparse(seed) for seed in seeds])
    n_threads = study["n_threads"]
    threads = list()
    for t in range(n_threads):
        pf = PageFetcher(scheduler)
        pf.start()
        threads.append(pf)
    for thread in threads:
        thread.join()
    t = time.time() - t
    print(f"{t} segundos = f{t / 60} minutos")
    results.append({"study": study, "seed": seed, "time": t})

  0%|                                                     | 0/8 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame.from_dict(results)
df

In [ ]:
df.to_parquet("results.parquet")